<a href="https://colab.research.google.com/github/codechief30/shopping-assistant/blob/main/MULTI_AGENT_PROJECT_SHOPPING_ASSISTANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain langchain-community transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install huggingface_hub


In [ ]:
!pip install sentence-transformers
#this model is installed for embedding models like for semantic, vector databases,etc. The features of smarter search and filtering are implemented using sentence-transformers models

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline as hf_pipeline_fn


#For testing purposes, using distil gpt 2, will switch to a better LLM after testing phase so making a code such that no major changes in token would be required if llm models are changed

def create_llm(model_name="distilgpt2", max_tokens_small=150, max_tokens_large=300):
  if "distil" in model_name.lower() or "small" in model_name.lower():
        max_new_tokens = max_tokens_small
  else:
    max_new_tokens = max_tokens_large

  hf_pipe = hf_pipeline_fn(
      task="text-generation",
      model=model_name,
      max_new_tokens=max_new_tokens,
      do_sample=True,
      temperature=0.3
    )
  return HuggingFacePipeline(pipeline=hf_pipe)
llm = create_llm(model_name="distilgpt2")

  #by default it will run with distill gpt 2 but in future if we change the model, the tokens will be updated as better mdoel can work with larger input of data at a time

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-3121569623.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=hf_pipe)


In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
import statistics

# LangChain components
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [ ]:
class Product:
    id: str
    title: str
    specs: Dict[str, Any] = field(default_factory=dict)
    prices: Dict[str, float] = field(default_factory=dict)
    reviews: List[Dict[str, Any]] = field(default_factory=list)

    def average_price(self) -> Optional[float]:
        return statistics.mean(self.prices.values()) if self.prices else None

    def average_rating(self) -> Optional[float]:
        if not self.reviews:
            return None
        return statistics.mean([r["rating"] for r in self.reviews])

class SessionMemory:
    """
    Shared scratchpad for all agents across a session.
    """
    def __init__(self):
        self.logs: List[str] = []
        self.products_seen: Dict[str, Product] = {}  # product_id -> Product
        self.rejected_ids: set = set()               # rejected product IDs
        self.query_history: List[str] = []          # keep track of user queries

    def add_log(self, text: str):
        self.logs.append(text)

    def save_products(self, products: List[Product]):
        for p in products:
            self.products_seen[p.id] = p

    def mark_rejected(self, product_id: str):
        self.rejected_ids.add(product_id)

    def get_products(self) -> List[Product]:
        return [p for pid, p in self.products_seen.items() if pid not in self.rejected_ids]

    def get_all_logs(self) -> List[str]:
        return self.logs

    def parse_followup(self, text: str) -> dict:
        """
        Parse follow-up commands like:
        reject <title>, filter <keyword>, add <query>, exit
        """
        text = text.lower().strip()
        if text.startswith("reject "):
            return {"action": "reject", "value": text[7:].strip()}
        if text.startswith("filter "):
            return {"action": "filter", "value": text[7:].strip()}
        if text.startswith("add "):
            return {"action": "add", "value": text[4:].strip()}
        if text == "exit":
            return {"action": "exit"}
        return {"action": "unknown", "value": text}


In [ ]:
'''class SearchAgent:
    """Returns candidate products for a query (mocked)."""
    def __init__(self, memory: Memory):
        self.memory = memory

    def run(self, query: str, max_results: int = 5) -> List[Product]:
        # MOCK: replace with actual scraping or catalog lookup later
        catalog = [
            Product(id="A", title="Laptop A", specs={"Brand": "Dell", "RAM": "8GB"}),
            Product(id="B", title="Laptop B", specs={"Brand": "HP", "RAM": "16GB"}),
            Product(id="C", title="Laptop C", specs={"Brand": "Lenovo", "RAM": "8GB"}),
            Product(id="D", title="Laptop D", specs={"Brand": "Asus", "RAM": "16GB"}),
            Product(id="E", title="Laptop E", specs={"Brand": "Acer", "RAM": "12GB"}),
        ]
        results = catalog[:max_results]
        self.memory.add(f"SearchAgent: query='{query}' -> {[p.title for p in results]}")
        return results

class PriceAgent:
    def __init__(self, memory: Memory):
        self.memory = memory

    def run(self, products: List[Product], budget: float) -> List[Product]:
        for p in products:
            url = f"https://dummyjson.com/products/{p.id}"
            data = requests.get(url).json()

            # Assign price from API
            p.prices = {"FakeStoreAPI": data['price']}

            # <-- DEBUG print to check fetched prices
            print(f"DEBUG: {p.title} price = {data['price']}")

        self.memory.add("PriceAgent: prices fetched from API")
        return products

class ReviewAgent:
    """Generates mock reviews/ratings and stores them on products."""
    def __init__(self, memory: Memory):
        self.memory = memory

    def run(self, products: List[Product]) -> List[Product]:
        for p in products:
            # MOCK: five faux reviews with ratings 1-5
            p.reviews = [{"rating": random.randint(1, 5), "text": f"Review {i}"} for i in range(5)]
        self.memory.add("ReviewAgent: generated dummy reviews")
        return products'''

'class SearchAgent:\n    """Returns candidate products for a query (mocked)."""\n    def __init__(self, memory: Memory):\n        self.memory = memory\n\n    def run(self, query: str, max_results: int = 5) -> List[Product]:\n        # MOCK: replace with actual scraping or catalog lookup later\n        catalog = [\n            Product(id="A", title="Laptop A", specs={"Brand": "Dell", "RAM": "8GB"}),\n            Product(id="B", title="Laptop B", specs={"Brand": "HP", "RAM": "16GB"}),\n            Product(id="C", title="Laptop C", specs={"Brand": "Lenovo", "RAM": "8GB"}),\n            Product(id="D", title="Laptop D", specs={"Brand": "Asus", "RAM": "16GB"}),\n            Product(id="E", title="Laptop E", specs={"Brand": "Acer", "RAM": "12GB"}),\n        ]\n        results = catalog[:max_results]\n        self.memory.add(f"SearchAgent: query=\'{query}\' -> {[p.title for p in results]}")\n        return results\n\nclass PriceAgent:\n    def __init__(self, memory: Memory):\n        self.mem

In [ ]:
import json
#To parse structured JSON responses from the LLM.
from typing import List, Dict, Any
class CoordinatorAgent:
    def __init__(self, memory: SessionMemory, llm, budget: float):
        self.memory = memory
        self.llm = llm
        self.budget = budget
        # prompt template for the LLM to decide
        self.prompt = PromptTemplate(
            input_variables=["products_info", "budget"],
            template=(
                "You are a shopping assistant. Here are candidate products:\n\n"
                "{products_info}\n\n"
                "User budget: ₹{budget}\n\n"
                "From these, choose the best product considering price and reviews. "
                "Give the recommended product title, its key reasons (short bullet points), "
                "and a short explanation for why other options were not chosen."
                "Respond in valid JSON format with keys: title, reasons[], explanation."
            )
        )
        #the prompt is the prompt to llm, asking it to provide a json file with keys of all the budget, and other user reuierments.

    def run(self, products: List[Product]) -> Dict[str, Any]:
        # Filter by average price <= budget

        affordable = [p for p in products if p.average_price() is not None and p.average_price() <= self.budget and p.id not in self.memory.rejected_ids]
        if not affordable:
            msg = f"No products are within the budget ₹{self.budget}."
            self.memory.add_log(f"CoordinatorAgent: {msg}")
            return {"recommendation": None, "explanation": msg}
        products_info_lines=[]
        for p in affordable:
          products_info_lines.append(
              f"-{p.title}|specs:{p.specs}|avg_price:₹{p.average_price():.2f}|avg_rating:{p.average_rating():.2f}"
          )
        products_info="\n".join(products_info_lines)

        #Creating an LLm chain to get decision text
        chain = LLMChain(llm=self.llm, prompt=self.prompt)
        decision_text = chain.run(products_info=products_info, budget=str(self.budget))
        try:
          parsed = json.loads(decision_text)
          return parsed
        except Exception:
          return {"recommendation": decision_text, "explanation": "Raw text"}


        # Save to memory and return
        self.memory.add_log("CoordinatorAgent: decision obtained from LLM")
        self.memory.add_log(f"CoordinatorAgent decision: {decision_text[:300]}")  # store first 300 chars for brevity
        return result


In [ ]:
'''from langchain.chains import LLMChain
llm = create_llm("distilgpt2")
# Models + Memory
@dataclass
# dataclass is a decorator used so we do not need to define innit function again and again in our corresponding classes

class Product:
    id: str
    title: str
    specs: Dict[str, Any] = field(default_factory=dict)
    #ensuring that every product gets its own empty dictionary
    prices: Dict[str, float] = field(default_factory=dict)
    reviews: List[Dict[str, Any]] = field(default_factory=list)


    def average_price(self) -> Optional[float]:
        return statistics.mean(self.prices.values()) if self.prices else None


    def average_rating(self) -> Optional[float]:
      return statistics.mean([r["rating"] for r in self.reviews]) if self.reviews else None

class SearchAgent:
    def __init__(self, memory: Memory):
        self.memory = memory
    def run(self, query: str, max_results: int = 5) -> List[Product]:
        catalog = [
            Product(id="A", title="Laptop A", specs={"Brand": "Dell", "RAM": "8GB"}),
            Product(id="B", title="Laptop B", specs={"Brand": "HP", "RAM": "16GB"}),
            Product(id="C", title="Laptop C", specs={"Brand": "Lenovo", "RAM": "8GB"}),
            Product(id="D", title="Laptop D", specs={"Brand": "Asus", "RAM": "16GB"}),
            Product(id="E", title="Laptop E", specs={"Brand": "Acer", "RAM": "12GB"}),
        ]
        results = catalog[:max_results]
        self.memory.add(f"SearchAgent: query='{query}' results={[p.title for p in results]}")
        return results

class SessionMemory:
    """
    A single scratchpad / session store for all agents.
    """
    def __init__(self):
        self.logs: List[str] = []
        self.products_seen: Dict[str, Product] = {}  # product_id -> Product
        self.rejected_ids: set = set()               # track rejected
        self.query_history: List[str] = []

    def add_log(self, text: str):
        self.logs.append(text)

    def save_products(self, products: List[Product]):
        for p in products:
            self.products_seen[p.id] = p

    def mark_rejected(self, product_id: str):
        self.rejected_ids.add(product_id)

    def get_products(self):
        return list(self.products_seen.values())

#PRICE AGENT CODE FOR MOCK DATABASE
class PriceAgent:
    def __init__(self, memory: Memory):
        self.memory = memory
    def run(self, products: List[Product], budget: float) -> List[Product]:
        for p in products:
            p.prices = {
                "Amazon": round(random.uniform(budget * 0.75, budget * 1.25), 2),
                "Flipkart": round(random.uniform(budget * 0.7, budget * 1.3), 2),
            }
        self.memory.add("PriceAgent: prices assigned")
        return products

#REVIEW AGENT CODE FOR MOCK

class ReviewAgent:
    def __init__(self, memory: Memory):
        self.memory = memory
    def run(self, products: List[Product]) -> List[Product]:
        for p in products:
            p.reviews = [{"rating": random.randint(1, 5), "text": f"Sample review {i}"} for i in range(5)]
        self.memory.add("ReviewAgent: reviews assigned")
        return products'''

'from langchain.chains import LLMChain\nllm = create_llm("distilgpt2")\n# Models + Memory\n@dataclass\n# dataclass is a decorator used so we do not need to define innit function again and again in our corresponding classes\n\nclass Product:\n    id: str\n    title: str\n    specs: Dict[str, Any] = field(default_factory=dict)\n    #ensuring that every product gets its own empty dictionary\n    prices: Dict[str, float] = field(default_factory=dict)\n    reviews: List[Dict[str, Any]] = field(default_factory=list)\n\n\n    def average_price(self) -> Optional[float]:\n        return statistics.mean(self.prices.values()) if self.prices else None\n\n\n    def average_rating(self) -> Optional[float]:\n      return statistics.mean([r["rating"] for r in self.reviews]) if self.reviews else None\n\nclass SearchAgent:\n    def __init__(self, memory: Memory):\n        self.memory = memory\n    def run(self, query: str, max_results: int = 5) -> List[Product]:\n        catalog = [\n            Product

In [ ]:
from langchain.chains import LLMChain
llm = create_llm("distilgpt2")
# Models + Memory
@dataclass
# dataclass is a decorator used so we do not need to define innit function again and again in our corresponding classes

class Product:
    id: str
    title: str
    specs: Dict[str, Any] = field(default_factory=dict)
    #ensuring that every product gets its own empty dictionary
    prices: Dict[str, float] = field(default_factory=dict)
    reviews: List[Dict[str, Any]] = field(default_factory=list)


    def average_price(self) -> Optional[float]:
        return statistics.mean(self.prices.values()) if self.prices else None


    def average_rating(self) -> Optional[float]:
      return statistics.mean([r["rating"] for r in self.reviews]) if self.reviews else None



# Agents
#in the initial code the agents were using the mockdatasets as their database, now i have implemented one api called Fake Store API.
#commenting down the codes of initial stages in case, the api key does not work.
#SEARCH AGENT CODE FOR MOCK DATABASE

#to treat the follow up requests of the user


#SEARCH AGENT CODE FOR API

import requests

class SearchAgent:
    def __init__(self, memory: SessionMemory):
        self.memory = memory

    def run(self, query: str, max_results: int = 5) -> List[Product]:
        self.memory.query_history.append(query)
        url = "https://dummyjson.com/products/search"
        try:
          req = requests.get(url, params={"q": query}, timeout=5)
          req.raise_for_status()
          data = req.json()
        except Exception as e:
          self.memory.add_log(f"SearchAgent API error: {e}")
          return []
        filtered = data['products'][:max_results]
        products = []
        for p in filtered:
            if str(p['id']) in self.memory.rejected_ids:
                continue  # skip rejected
            products.append(Product(
                id=str(p['id']),
                title=p['title'],
                specs={"category": p['category']}
            ))
        self.memory.save_products(products)
        self.memory.add_log(f"SearchAgent: query='{query}' -> {[p.title for p in products]}")
        return products


#USING API, PRICE AGENT CODE
class PriceAgent:
    def __init__(self, memory: SessionMemory):
        self.memory = memory

    def run(self, products: List[Product], budget: float) -> List[Product]:
        for p in products:
            if p.id in self.memory.rejected_ids:  # skip rejected
                continue
            data=requests.get(f"https://dummyjson.com/products/{p.id}").json()
            p.prices={"FakeStoreAPI":data['price']}
            try:
              req = requests.get(f"https://dummyjson.com/products/{p.id}", timeout=5)
              req.raise_for_status()
              data=req.json()
              rating=data.get("rating",0)
              p.reviews=[{"rating":round(rating),"text":f"Sample Review {i+1} "} for i in range (3)]

            except Exception as e:
              self.memory.add_log(f"Review API Agent Error for product {p.id}: {e}")
              p.reviews=[{"ratings": 0, "text": "No reviews available"}]

        self.memory.save_products(products)
        self.memory.add_log("PriceAgent: prices fetched from API")
        return products



#REVIEW AGENT CODE OF API
class ReviewAgent:
    def __init__(self, memory: SessionMemory):
        self.memory = memory

    def run(self, products: List[Product]) -> List[Product]:
        for p in products:
            if p.id in self.memory.rejected_ids:
                continue
            data = requests.get(f"https://dummyjson.com/products/{p.id}").json()
            rating = data['rating']
            p.reviews = [{"rating": round(rating), "text": f"Sample review {i+1}"} for i in range(3)]
        self.memory.save_products(products)
        self.memory.add_log("ReviewAgent: reviews fetched from API")
        return products


#coordinator agent and feedback loop will remain the same and unchanged because, it works with real time price now

#In the updated code:
#works on the full set of products seen so far (not only the last batch) and filters out items marked rejected.


#updated code for feeback loop
#built the “shared scratchpad” version using SessionMemory (Python object + dictionaries).
#We solved the “no scoped memory” issue by:
#Creating a persistent SessionMemory object.
#Passing that same object to every agent.
#Having agents write their intermediate results (products, rejections, reviews) into that memory instead of throwing them away.

Device set to use cpu


In [ ]:
session_memory = SessionMemory()
#created one session memory through which the agents will talk
def shopping_assistant_pipeline(user_query: str, budget: float):
    """Full pipeline with feedback from PriceAgent and ReviewAgent."""
    search_agent = SearchAgent(session_memory)
    price_agent = PriceAgent(session_memory)
    review_agent = ReviewAgent(session_memory)
    coordinator = CoordinatorAgent(session_memory, llm, budget)

    # --- Step 1: initial search ---
    products = search_agent.run(user_query)
    products = price_agent.run(products, budget)
    products = review_agent.run(products)

    # --- Feedback from PriceAgent to SearchAgent ---
    valid_prices = [p.average_price() for p in products if p.average_price() is not None]
    if valid_prices and all(price > budget for price in valid_prices):
        print("All results exceed budget. Trying cheaper models automatically...")
        cheaper_query = user_query + " cheap"
        products = search_agent.run(cheaper_query)
        products = price_agent.run(products, budget)
        products = review_agent.run(products)

    # --- Feedback from ReviewAgent to SearchAgent ---
    valid_ratings = [p.average_rating() for p in products if p.average_rating() is not None]
    if valid_ratings and all(rating < 3 for rating in valid_ratings):
        print("All results poorly rated. Searching for better reviews automatically...")
        better_query = user_query + " highly rated"
        products = search_agent.run(better_query)
        products = price_agent.run(products, budget)
        products = review_agent.run(products)

    # --- Coordinator runs ---
    recommendation = coordinator.run(products)

    return products, recommendation


In [ ]:

# Step 1: Greet and collect inputs
print("👋 Welcome to your Shopping Assistant!")
name = input("May I know your name? ")
print(f"Hi {name}! Let's find the best products for you.")

user_query = input("What would you like me to suggest today? ")
budget = float(input("Please enter your budget (₹): "))


coordinator=CoordinatorAgent(session_memory,llm,budget)


# Run pipeline
products, result = shopping_assistant_pipeline(user_query, budget)

# Show product list
print("\nHere are some product suggestions for you:")
for p in products:
    print(f"- {p.title} | Avg Price: ₹{p.average_price()} | Avg Rating: {p.average_rating()}")

# Show recommendation
print("\n=== FIRST RECOMMENDATION ===")
print(result)


# Step 4: Interactive refinement loop
while True:
    user_feedback = input("\nWould you like to refine your request? (type 'reject <title>' or 'show <query>' or 'exit'): ")

    if user_feedback.lower().startswith("reject "):
        title = user_feedback[7:].strip()
        for pid, prod in session_memory.products_seen.items():
            if prod.title.lower() == title.lower():
                session_memory.mark_rejected(pid)
        products = [p for p in session_memory.get_products() if p.id not in session_memory.rejected_ids]
        result = coordinator.run(products)
        print("\nUpdated Recommendation:", result)
        print("\n Here are the updated product sessions for you: ")
        for p in products:
          print(f"-{p.title} | Avg Price: ₹{p.average_price()} | Avg Rating: {p.average_rating()}")

    elif user_feedback.lower().startswith("show "):
        new_query = user_feedback[5:].strip()
        products, result = shopping_assistant_pipeline(new_query, budget)
        for p in products:
            print(f"- {p.title} | Avg Price: ₹{p.average_price()} | Avg Rating: {p.average_rating()}")
        print("\nCurrent Recommendation:", result)



    elif user_feedback.lower() == "exit":
        print("\nThanks for using your Shopping Assistant! Happy shopping 🛍️")
        break

# Example
#shopping_assistant_pipeline("shirt", 10000)

👋 Welcome to your Shopping Assistant!
May I know your name? Abhi
Hi Abhi! Let's find the best products for you.
What would you like me to suggest today? laptop
Please enter your budget (₹): 100000


/tmp/ipython-input-1824332100.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt)
/tmp/ipython-input-1824332100.py:41: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  decision_text = chain.run(products_info=products_info, budget=str(self.budget))
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Here are some product suggestions for you:
- Apple MacBook Pro 14 Inch Space Grey | Avg Price: ₹1999.99 | Avg Rating: 4
- Asus Zenbook Pro Dual Screen Laptop | Avg Price: ₹1799.99 | Avg Rating: 4
- Huawei Matebook X Pro | Avg Price: ₹1399.99 | Avg Rating: 5
- Lenovo Yoga 920 | Avg Price: ₹1099.99 | Avg Rating: 3
- New DELL XPS 13 9300 Laptop | Avg Price: ₹1499.99 | Avg Rating: 3

=== FIRST RECOMMENDATION ===
{'recommendation': "You are a shopping assistant. Here are candidate products:\n\n-Apple MacBook Pro 14 Inch Space Grey|specs:{'category': 'laptops'}|avg_price:₹1999.99|avg_rating:4.00\n-Asus Zenbook Pro Dual Screen Laptop|specs:{'category': 'laptops'}|avg_price:₹1799.99|avg_rating:4.00\n-Huawei Matebook X Pro|specs:{'category': 'laptops'}|avg_price:₹1399.99|avg_rating:5.00\n-Lenovo Yoga 920|specs:{'category': 'laptops'}|avg_price:₹1099.99|avg_rating:3.00\n-New DELL XPS 13 9300 Laptop|specs:{'category': 'laptops'}|avg_price:₹1499.99|avg_rating:3.00\n\nUser budget: ₹100000.0\n\nFro

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Updated Recommendation: {'recommendation': "You are a shopping assistant. Here are candidate products:\n\n-Apple MacBook Pro 14 Inch Space Grey|specs:{'category': 'laptops'}|avg_price:₹1999.99|avg_rating:4.00\n-Asus Zenbook Pro Dual Screen Laptop|specs:{'category': 'laptops'}|avg_price:₹1799.99|avg_rating:4.00\n-Lenovo Yoga 920|specs:{'category': 'laptops'}|avg_price:₹1099.99|avg_rating:3.00\n-New DELL XPS 13 9300 Laptop|specs:{'category': 'laptops'}|avg_price:₹1499.99|avg_rating:3.00\n\nUser budget: ₹100000.0\n\nFrom these, choose the best product considering price and reviews. Give the recommended product title, its key reasons (short bullet points), and a short explanation for why other options were not chosen.Respond in valid JSON format with keys: title, reasons[], explanation.\n-New DELL XPS 13 9300 Laptop|specs:{'category': 'laptops'}|avg_price:₹1499.99|avg_rating:3.00\n-New DELL XPS 13 9300 Laptop|specs:{'category': 'laptops'}|avg_price:₹1499.99|avg_rating:3.00\n-New DELL XPS 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- Apple iPhone Charger | Avg Price: ₹19.99 | Avg Rating: 4
- Apple MagSafe Battery Pack | Avg Price: ₹99.99 | Avg Rating: 4
- iPhone 12 Silicone Case with MagSafe Plum | Avg Price: ₹29.99 | Avg Rating: 4
- Selfie Lamp with iPhone | Avg Price: ₹14.99 | Avg Rating: 4
- iPhone 5s | Avg Price: ₹199.99 | Avg Rating: 3

Current Recommendation: {'recommendation': "You are a shopping assistant. Here are candidate products:\n\n-Apple iPhone Charger|specs:{'category': 'mobile-accessories'}|avg_price:₹19.99|avg_rating:4.00\n-Apple MagSafe Battery Pack|specs:{'category': 'mobile-accessories'}|avg_price:₹99.99|avg_rating:4.00\n-iPhone 12 Silicone Case with MagSafe Plum|specs:{'category': 'mobile-accessories'}|avg_price:₹29.99|avg_rating:4.00\n-Selfie Lamp with iPhone|specs:{'category': 'mobile-accessories'}|avg_price:₹14.99|avg_rating:4.00\n-iPhone 5s|specs:{'category': 'smartphones'}|avg_price:₹199.99|avg_rating:3.00\n\nUser budget: ₹100000.0\n\nFrom these, choose the best product considering pric